For Devs

Figuring out how to create example vids


In [2]:
from typing import Any, List,Collection,Dict,Tuple
import cv2
import numpy as np
import os
import time
import bsoid_ipynb_header  # Ensures that the bsoid module can found no matter what computer you use. Not necessary anywhere else.
import bsoid
# from bsoid import check_arg, logging_bsoid, config, statistics
logger = bsoid.config.initialize_logger('Pipelin-createVideoExamples')

In [3]:
### Instantiating data file paths
pipeline_name = 'example1.pipeline'
pipeline_file_path = os.path.join(bsoid.config.BSOID_BASE_PROJECT_PATH, 'output', pipeline_name)
assert os.path.isfile(pipeline_file_path)

In [4]:
p = bsoid.read_pipeline(pipeline_file_path)

2020-11-15 10:54:31,235 - C:\Users\killian\projects\B-SOID\bsoid\io.py               - DEBUG    - read_pipeline(): Trying to open: C:\Users\killian\projects\B-SOID\output\example1.pipeline
2020-11-15 10:54:31,281 - C:\Users\killian\projects\B-SOID\bsoid\io.py               - DEBUG    - read_pipeline(): Pipeline at C:\Users\killian\projects\B-SOID\output\example1.pipeline opened successfully!


In [5]:
# Get data set where it is predicted by the model
df = p.df_features_predict.copy()
df.head()

,DistFrontPawsTailbaseRelativeBodyLength,DistBackPawsBaseTailRelativeBodyLength,InterforepawDistance,BodyLength,SnoutToTailbaseChangeInAngle,SnoutSpeed,TailbaseSpeed,scorer,frame,file_source,data_source
0,29.330171,83.812585,25.397233,116.828076,0.137147,1349.824748,1322.839091,DLC_resnet50_EPM_DLC_BSOIDAug25shuffle1_495000,0,C:\Users\killian\projects\B-SOID\example_data\Video4DLC_resnet50_EPM_DLC_BSOIDAug25shuffle1_495000.csv,Video4DLC_resnet50_EPM_DLC_BSOIDAug25shuffle1_495000.csv
1,36.448049,102.701161,31.112057,144.353813,0.323085,843.947896,827.969228,DLC_resnet50_EPM_DLC_BSOIDAug25shuffle1_495000,1,C:\Users\killian\projects\B-SOID\example_data\Video4DLC_resnet50_EPM_DLC_BSOIDAug25shuffle1_495000.csv,Video4DLC_resnet50_EPM_DLC_BSOIDAug25shuffle1_495000.csv
2,41.526207,114.886229,34.781627,163.183003,0.517633,338.072408,333.079183,DLC_resnet50_EPM_DLC_BSOIDAug25shuffle1_495000,2,C:\Users\killian\projects\B-SOID\example_data\Video4DLC_resnet50_EPM_DLC_BSOIDAug25shuffle1_495000.csv,Video4DLC_resnet50_EPM_DLC_BSOIDAug25shuffle1_495000.csv
3,41.493481,114.023197,34.497520,162.972212,0.740347,0.817159,2.844259,DLC_resnet50_EPM_DLC_BSOIDAug25shuffle1_495000,3,C:\Users\killian\projects\B-SOID\example_data\Video4DLC_resnet50_EPM_DLC_BSOIDAug25shuffle1_495000.csv,Video4DLC_resnet50_EPM_DLC_BSOIDAug25shuffle1_495000.csv
4,40.868735,113.043302,34.209630,161.685544,0.964092,0.956624,3.129313,DLC_resnet50_EPM_DLC_BSOIDAug25shuffle1_495000,4,C:\Users\killian\projects\B-SOID\example_data\Video4DLC_resnet50_EPM_DLC_BSOIDAug25shuffle1_495000.csv,Video4DLC_resnet50_EPM_DLC_BSOIDAug25shuffle1_495000.csv


In [11]:
df = p.df_features_train_scaled.copy()
df.head(5)

,index,DistFrontPawsTailbaseRelativeBodyLength,DistBackPawsBaseTailRelativeBodyLength,InterforepawDistance,BodyLength,SnoutToTailbaseChangeInAngle,SnoutSpeed,TailbaseSpeed,scorer,frame,file_source,data_source,dim_1,dim_2,dim_3,gmm_assignment,is_test_data,svm_assignment
0,21788,0.891293,0.755149,1.084600,0.521937,0.637257,0.066744,0.485897,DLC_resnet50_EPM_DLC_BSOIDAug25shuffle1_495000,2937,C:\Users\killian\projects\B-SOID\example_data\Video3DLC_resnet50_EPM_DLC_BSOIDAug25shuffle1_495000.csv,Video3DLC_resnet50_EPM_DLC_BSOIDAug25shuffle1_495000.csv,-16.751085,3.619697,18.670547,12,True,12
1,27907,-0.590306,-0.075692,0.118740,-0.178832,0.030800,-0.282731,-0.121996,DLC_resnet50_EPM_DLC_BSOIDAug25shuffle1_495000,9056,C:\Users\killian\projects\B-SOID\example_data\Video3DLC_resnet50_EPM_DLC_BSOIDAug25shuffle1_495000.csv,Video3DLC_resnet50_EPM_DLC_BSOIDAug25shuffle1_495000.csv,-19.821964,-11.579646,-34.659081,1,True,1
2,11873,0.883916,1.044396,0.469022,1.004911,-0.051985,0.141922,0.325390,DLC_resnet50_EPM_DLC_BSOIDAug25shuffle1_495000,2387,C:\Users\killian\projects\B-SOID\example_data\Video2DLC_resnet50_EPM_DLC_BSOIDAug25shuffle1_495000.csv,Video2DLC_resnet50_EPM_DLC_BSOIDAug25shuffle1_495000.csv,4.103022,7.387649,23.261028,3,True,3
3,20245,1.215963,0.544161,0.237078,0.303330,0.017370,-0.176382,-0.122246,DLC_resnet50_EPM_DLC_BSOIDAug25shuffle1_495000,1394,C:\Users\killian\projects\B-SOID\example_data\Video3DLC_resnet50_EPM_DLC_BSOIDAug25shuffle1_495000.csv,Video3DLC_resnet50_EPM_DLC_BSOIDAug25shuffle1_495000.csv,-33.721294,25.904421,-2.396084,7,True,7
4,14008,0.543854,-0.512256,0.290913,-0.737650,-0.022857,-0.235316,-0.114827,DLC_resnet50_EPM_DLC_BSOIDAug25shuffle1_495000,4522,C:\Users\killian\projects\B-SOID\example_data\Video2DLC_resnet50_EPM_DLC_BSOIDAug25shuffle1_495000.csv,Video2DLC_resnet50_EPM_DLC_BSOIDAug25shuffle1_495000.csv,-21.918512,-10.120231,36.819775,12,True,12


In [12]:
# Find out the total seconds of runtime in the video
fps = 30
runtime_seconds = len(df) / fps
runtime_seconds

945.6666666666666

In [17]:
# Calculate the duration, in seconds, that each behaviour runs.
duration_seconds_per_assignment = {}
data = list(df[p.svm_assignment_col_name].values)  # Ex: [1,2,3,3,3,3,4 .....]
for assignment in data:
    if assignment not in duration_seconds_per_assignment:
        duration_seconds_per_assignment[assignment] = 1/fps
    else:
        duration_seconds_per_assignment[assignment] = duration_seconds_per_assignment[assignment] + 1/fps
duration_seconds_per_assignment

{12: 44.16666666666672,
 1: 58.366666666665914,
 3: 63.43333333333229,
 7: 52.63333333333291,
 14: 59.63333333333251,
 2: 55.26666666666609,
 6: 61.56666666666573,
 0: 75.89999999999826,
 9: 56.799999999999336,
 4: 66.13333333333215,
 5: 62.333333333332355,
 15: 67.19999999999875,
 11: 48.99999999999978,
 13: 70.79999999999855,
 10: 54.63333333333279,
 8: 47.79999999999985}

In [18]:
duration_percent_of_each_assignment = {ass: dur/73 for ass, dur in duration_seconds_per_assignment.items()}
duration_percent_of_each_assignment

{12: 0.605022831050229,
 1: 0.7995433789954235,
 3: 0.8689497716894835,
 7: 0.7210045662100398,
 14: 0.8168949771689384,
 2: 0.7570776255707684,
 6: 0.8433789954337771,
 0: 1.0397260273972364,
 9: 0.7780821917808128,
 4: 0.9059360730593444,
 5: 0.8538812785387994,
 15: 0.9205479452054623,
 11: 0.6712328767123258,
 13: 0.9698630136986103,
 10: 0.7484018264840109,
 8: 0.6547945205479431}